In [2]:
import pandas as pd

data=pd.read_csv("fil_data_movies_lessthan10k_replaced.csv")

In [6]:
data.head

<bound method NDFrame.head of            id                                   title  vote_average   
0       27205                               Inception         8.364  \
1      157336                            Interstellar         8.417   
2         155                         The Dark Knight         8.512   
3       19995                                  Avatar         7.573   
4       24428                            The Avengers         7.710   
...       ...                                     ...           ...   
26224  656353       John Was Trying to Contact Aliens         6.420   
26225  288706  Italy in a Day - Un giorno da italiani         7.750   
26226  312152               Chuck Norris vs Communism         7.100   
26227   41769     Prom Night IV: Deliver Us from Evil         4.340   
26228  480626                   The Christmas Cottage         6.441   

       vote_count release_date     revenue  runtime  adult   
0           34495    7/15/2010   825532764      148  Fa